In [1]:
import polars as pl
import os

train_applprev_1_0 = pl.read_csv("Data/csv_files/train/train_applprev_1_0.csv")
train_applprev_1_1 = pl.read_csv("Data/csv_files/train/train_applprev_1_1.csv")

In [2]:
train_applprev_1 = pl.concat([train_applprev_1_0, train_applprev_1_1])

directory = r"C:\Users\afise\.git\CreditRiskModel\Merged_Data"
filename = "train_applprev_1.csv"
file_path = os.path.join(directory, filename)

train_applprev_1.write_csv(file_path)

In [3]:
unique_case_id_count = train_applprev_1.select(pl.col("case_id").unique().count())
print(unique_case_id_count)

shape: (1, 1)
┌─────────┐
│ case_id │
│ ---     │
│ u32     │
╞═════════╡
│ 1221522 │
└─────────┘


In [4]:
train_applprev_1.shape

(6525979, 41)

In [5]:
df_lazy = pl.scan_csv(file_path)

In [6]:
import polars as pl

def try_parse_date(col, fmt1, fmt2):
    date1 = col.str.strptime(pl.Date, fmt1, strict=False)
    date2 = col.str.strptime(pl.Date, fmt2, strict=False)
    return pl.when(date1.is_not_null()).then(date1).otherwise(date2)

df_grouped = (
    df_lazy
    .with_columns([
        try_parse_date(pl.col("approvaldate_319D"), "%m/%d/%Y", "%Y-%m-%d").alias("approvaldate_319D"),
        try_parse_date(pl.col("dateactivated_425D"), "%m/%d/%Y", "%Y-%m-%d").alias("dateactivated_425D"),
        try_parse_date(pl.col("creationdate_885D"), "%m/%d/%Y", "%Y-%m-%d").alias("creationdate_885D"),
        try_parse_date(pl.col("dtlastpmt_581D"), "%m/%d/%Y", "%Y-%m-%d").alias("dtlastpmt_581D"),
        try_parse_date(pl.col("employedfrom_700D"), "%m/%d/%Y", "%Y-%m-%d").alias("employedfrom_700D"),
    ])
    .groupby("case_id")
    .agg([
        pl.col("actualdpd_943P").mean().alias("actualdpd_943P_mean"),
        pl.col("annuity_853A").sum().alias("annuity_853A_sum"),
        pl.col("childnum_21L").sum().alias("childnum_21L_sum"),
        pl.col("credacc_actualbalance_314A").mean().alias("credacc_actualbalance_314A_mean"),
        pl.col("credacc_credlmt_575A").mean().alias("credacc_credlmt_575A_mean"),
        pl.col("credacc_maxhisbal_375A").max().alias("credacc_maxhisbal_375A_max"),
        pl.col("credacc_minhisbal_90A").min().alias("credacc_minhisbal_90A_min"),
        pl.col("credacc_transactions_402L").sum().alias("credacc_transactions_402L_sum"),
        pl.col("credamount_590A").mean().alias("credamount_590A_mean"),
        pl.col("currdebt_94A").mean().alias("currdebt_94A_mean"),
        pl.col("downpmt_134A").sum().alias("downpmt_134A_sum"),
        pl.col("mainoccupationinc_437A").mean().alias("mainoccupationinc_437A_mean"),
        pl.col("outstandingdebt_522A").sum().alias("outstandingdebt_522A_sum"),
        pl.col("pmtnum_8L").max().alias("pmtnum_8L_max"),
        pl.col("tenor_203L").min().alias("tenor_203L_min"),
        pl.col("isbidproduct_390L").cast(pl.UInt32).sum().alias("isbidproduct_390L_sum"),
        pl.col("isdebitcard_527L").cast(pl.UInt32).sum().alias("isdebitcard_527L_sum"),
        pl.col("credacc_status_367L").mode().alias("credacc_status_367L_mode"),
        pl.col("credtype_587L").mode().alias("credtype_587L_mode"),
        pl.col("education_1138M").mode().alias("education_1138M_mode"),
        pl.col("familystate_726L").mode().alias("familystate_726L_mode"),
        pl.col("postype_4733339M").mode().alias("postype_4733339M_mode"),
        pl.col("profession_152M").mode().alias("profession_152M_mode"),
        pl.col("rejectreason_755M").mode().alias("rejectreason_755M_mode"),
        pl.col("rejectreasonclient_4145042M").mode().alias("rejectreasonclient_4145042M_mode"),
        pl.col("status_219L").mode().alias("status_219L_mode"),
        (pl.col("approvaldate_319D").diff().abs().min()).alias("approval_to_activation_min_diff"),
        (pl.col("creationdate_885D").diff().abs().min()).alias("creation_min_diff"),
        (pl.col("dtlastpmt_581D").diff().abs().max()).alias("payment_max_diff"),
        pl.col("employedfrom_700D").min().alias("earliest_employment_date"),
    ])
    .collect()
)

C:\Users\afise\AppData\Local\Temp\ipykernel_21436\3554112918.py:17: DeprecationWarning: `groupby` is deprecated. It has been renamed to `group_by`.
  .groupby("case_id")


In [7]:
df_grouped.head()

case_id,actualdpd_943P_mean,annuity_853A_sum,childnum_21L_sum,credacc_actualbalance_314A_mean,credacc_credlmt_575A_mean,credacc_maxhisbal_375A_max,credacc_minhisbal_90A_min,credacc_transactions_402L_sum,credamount_590A_mean,currdebt_94A_mean,downpmt_134A_sum,mainoccupationinc_437A_mean,outstandingdebt_522A_sum,pmtnum_8L_max,tenor_203L_min,isbidproduct_390L_sum,isdebitcard_527L_sum,credacc_status_367L_mode,credtype_587L_mode,education_1138M_mode,familystate_726L_mode,postype_4733339M_mode,profession_152M_mode,rejectreason_755M_mode,rejectreasonclient_4145042M_mode,status_219L_mode,approval_to_activation_min_diff,creation_min_diff,payment_max_diff,earliest_employment_date
i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,u32,u32,list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],duration[ms],duration[ms],duration[ms],date
1314493,0.0,27415.4001,0.0,null,0.0,null,null,0.0,23862.769231,0.0,0.0,20173.846154,0.0,24.0,3.0,0,0,[null],"[""CAL""]","[""P97_36_170""]","[""MARRIED""]","[""a55475b1""]","[""a55475b1""]","[""a55475b1""]","[""a55475b1""]","[""D""]",258d,0ms,null,2012-04-15
1713622,0.0,42863.4001,0.0,null,0.0,null,null,0.0,102263.0,0.0,0.0,36285.714286,0.0,48.0,12.0,0,0,[null],"[""COL"", ""CAL""]","[""a55475b1""]",[null],"[""P177_117_192""]","[""a55475b1""]","[""P99_56_166""]","[""P94_109_143""]","[""D""]",null,0ms,null,2011-06-15
1498510,0.0,26889.4001,0.0,null,0.0,null,null,0.0,44266.666667,11001.786667,0.0,54166.666667,42509.6,24.0,6.0,0,0,[null],"[""CAL""]","[""P33_146_175""]","[""MARRIED"", ""SINGLE"", null]","[""a55475b1""]","[""a55475b1""]","[""a55475b1""]","[""a55475b1""]","[""D""]",366d,39d,null,2012-09-15
2582034,0.0,14963.6,1.0,null,0.0,null,null,0.0,31680.5,0.0,0.0,61000.0,0.0,18.0,7.0,0,0,[null],"[""COL""]","[""a55475b1""]",[null],"[""a55475b1""]","[""a55475b1""]","[""P94_109_143""]","[""P94_109_143""]","[""D""]",null,165d,null,2013-02-15
150749,0.0,21968.8001,0.0,null,0.0,null,null,0.0,20412.177778,1371.0249,9060.0,36133.333333,7383.6,24.0,6.0,1,0,[null],"[""COL""]","[""P97_36_170"", ""a55475b1""]","[""MARRIED""]","[""a55475b1""]","[""a55475b1""]","[""a55475b1""]","[""a55475b1""]","[""K"", ""D""]",242d,1d,null,2011-05-15


In [8]:
df_grouped.shape

(1221522, 31)

In [ ]:
import polars as pl

# Assuming df_grouped is your DataFrame after aggregation
# and mode_columns contains the names of the columns with list results from mode calculations

mode_columns = [
    "credacc_status_367L_mode", 
    "credtype_587L_mode",
    "education_1138M_mode",
    "familystate_726L_mode",
    "postype_4733339M_mode",
    "profession_152M_mode",
    "rejectreason_755M_mode",
    "rejectreasonclient_4145042M_mode",
    "status_219L_mode"
]

# Iterate through each mode column and update it
for column in mode_columns:
    df_grouped = df_grouped.with_columns(
        pl.when(
            pl.col(column).is_not_null() &
            (pl.apply(pl.col(column), lambda arr: len(arr)) > 0)
        )
        .then(pl.col(column).apply(lambda arr: arr[0]))
        .otherwise(pl.lit(None)).alias(column)
    )

# df_grouped now has the first mode value or None for each mode column

In [ ]:
# Assuming df_grouped is your dataframe with the aggregated data
df_filtered = df_grouped.filter(
    (pl.col("approval_to_activation_min_diff").is_not_null()) &
    (pl.col("creation_min_diff").is_not_null()) &
    (pl.col("payment_max_diff").is_not_null()) &
    (pl.col("earliest_employment_date").is_not_null())
)

# Display a sample of the filtered dataframe for inspection
df_filtered.describe()

In [ ]:
import duckdb

In [ ]:
csv_file_path = r"C:\Users\afise\.git\CreditRiskModel\Merged_Data\train_applprev_1.csv"

conn = duckdb.connect(database=':memory:', read_only=False)

conn.execute(f"CREATE TABLE train_applprev_1 AS SELECT * FROM read_csv_auto('{csv_file_path}')")

In [ ]:
query_result = conn.execute("SELECT * FROM train_applprev_1 LIMIT 10")
results = query_result.fetchall()
column_names = [description[0] for description in query_result.description]

print(', '.join(column_names))

for row in results:
    formatted_row = ', '.join([str(elem) if elem is not None else 'NULL' for elem in row])
    print(formatted_row)

In [ ]:
conn.close()